<a href="https://colab.research.google.com/github/sbchengyiwei/Berkeley-CS61B-fa20/blob/main/Submit_NEU_CS6220_Colab_3_1_Some_Map_Reduce_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://course.ccs.neu.edu/cs6220/homework-1/data/combined_data_1.txt

--2023-01-20 04:00:53--  https://course.ccs.neu.edu/cs6220/homework-1/data/combined_data_1.txt
Resolving course.ccs.neu.edu (course.ccs.neu.edu)... 129.10.117.35
Connecting to course.ccs.neu.edu (course.ccs.neu.edu)|129.10.117.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 495032008 (472M) [text/plain]
Saving to: ‘combined_data_1.txt’

combined_data_1.txt 100%[===================>] 472.10M  90.1MB/s    in 5.0s    

2023-01-20 04:00:58 (93.9 MB/s) - ‘combined_data_1.txt’ saved [495032008/495032008]



# The Average Star Rating for All Users




In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 499.6 kB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=2781a9741d22428f6d0ee463fcdef98664f5bbd81fcaa61732ae5ef7ab3d7064
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 23 not

In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

# Question

Write a function that finds the average star rating over users. 

* Output: `(<USER>,<AVERAGE RATINGS>)`

In [3]:
!head -n 10000 combined_data_1.txt > short_data.txt

In [34]:
lines = sc.textFile("short_data.txt", 1).filter(lambda x: ":" not in x).map(lambda x: x.split(","))
user_star_rating = lines.map(lambda x: (x[0],(float(x[1]), 1.0)))
result = user_star_rating.reduceByKey(lambda x,y : (x[0] + y[0], x[1] + y[1])).map(lambda x:(x[0], x[1][0]/x[1][1]))
result.take(10)


[('1488844', 3.5),
 ('822109', 5.0),
 ('885013', 4.5),
 ('30878', 2.5),
 ('823519', 2.0),
 ('893988', 3.0),
 ('124105', 4.0),
 ('1248029', 3.0),
 ('1842128', 4.0),
 ('2238063', 3.0)]

# Distribution of Users over Star Ratings

Find the distribution of number of users over star ratings.

* Output: `(<Number-Stars>, <TOTAL-NUMBER-USERS>)`


In [35]:
lines.map(lambda x: (x[1], 1)).reduceByKey(lambda x,y:x+y).collect()

[('3', 2450), ('5', 2190), ('4', 2926), ('2', 1260), ('1', 1166)]